# Один из возможных алгоритмов скрейпинга

* указать в коде адрес интересующего сайта: откуда вы хотите скачать данные?
для этого используется библиотека requests

* сохранить веб-страницу (html-код страницы)

* выбрать данные, которые нужно собрать (используется BeautifulSoup)

* записать данные в csv-файл.

Если нужно соскрейпить несколько страниц - повторяем процесс для каждой из них.

**Наша задача:** выгрузить недавние новости в датафрейм `pandas`, чтобы потом сохранить все в csv-файл.

Сначала сгрузим весь html-код страницы и сохраним его в отдельную переменную. Для этого нам понадобится библиотека `requests`. Документация: https://requests.readthedocs.io/en/latest/

Импортируем её:

In [1]:
import requests

Сохраним ссылку на главную страницу сайта в переменную `url` для удобства и выгрузим страницу. (Разумеется, это будет работать при подключении к интернету. Если соединение будет отключено, Python выдаст `NewConnectionError`).

In [2]:
url = 'https://nplus1.ru/' # сохраняем
page = requests.get(url) # загружаем страницу по ссылке

Если мы просто посмотрим на объект, мы ничего особенного не увидим:

In [3]:
page  # response 200 - страница загружена

<Response [200]>

**BeautifulSoup** - python-библиотека для синтаксического разбора файлов HTML/XML. В веб-разработке «суп из тегов» (tag soup) - это слово для синтаксически или структурно некорректного HTML, написанного для веб-страницы.

Документация: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

Импортируем функцию `BeautifulSoup` из библиотеки `bs4` (от *beautifulsoup4*) и заберём со страницы `page` код html в виде текста. 

Сохраним в переменную 'soup' весь HTML-код страницы. HTML-код - это "дерево тегов", формирующее контент страницы.

In [ ]:
!pip install bs4

In [4]:
from bs4 import BeautifulSoup 
# название - отсылка к песне про суп из Алисы в стране чудес https://aliceinwonderland.fandom.com/wiki/Turtle_Soup

In [6]:
soup = BeautifulSoup(page.text, 'html')

Если выведем `soup` на экран, мы увидим то же самое, что в режиме разработчика или в режиме происмотра исходного кода (`view-source` через *Ctrl+U* в Google Chrome).

In [7]:
soup

<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>N + 1 — главное издание о науке, технике и технологиях</title>
<link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/AeonikPro/AeonikPro-Regular.woff2" rel="preload" type="font/woff2"/>
<link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/Spectral/Spectral-Regular.woff" rel="preload" type="font/woff2"/>
<link href="/front-build/css/main.css?id=afa8ece698346e24e0ac250f09dc6e8f" rel="stylesheet"/>
<link href="/front-build/css/app.css?id=773a0b58c8d3c33a1082b6280d9ddc6d" rel="stylesheet"/>
<link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="/site.webmanifest" rel="manifest"/>
<link color="#f26e40" href=

Для просмотра выглядит не очень удобно.  «Причешем» наш `soup` – воспользуемся методом `.prettify()` в сочетании с функцией `print()`.

In [8]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   N + 1 — главное издание о науке, технике и технологиях
  </title>
  <link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/AeonikPro/AeonikPro-Regular.woff2" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/Spectral/Spectral-Regular.woff" rel="preload" type="font/woff2"/>
  <link href="/front-build/css/main.css?id=afa8ece698346e24e0ac250f09dc6e8f" rel="stylesheet"/>
  <link href="/front-build/css/app.css?id=773a0b58c8d3c33a1082b6280d9ddc6d" rel="stylesheet"/>
  <link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link href="/site.webmanifest" rel="manifest"

В такой выдаче ориентироваться гораздо удобнее (но при желании, то же можно увидеть в браузере, на большом экране).

Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`. 

**Функция `soup.find('a')`** найдет первый в дереве тег \<a>.
Если нам нужно найти не только первый элемент, а все элементы по определенному признаку, следует использовать функцию **`soup.find_all('a')`**

In [9]:
for link in soup.find_all('a'):
    print(link.get('href'))

/search
https://offline.nplus1.ru/
https://nplus.pro/
https://nplus1.ru/about
https://nplus1.ru/difficult
https://nplus1.ru/adv
https://nplus1.ru/blog/2022/04/01/samotek
/search/empty/869
/search/empty/874
/search/empty/880
/search/empty/768
/search/empty/890
/search/empty/871
/search/empty/876
/search/empty/775
/search/empty/767
/search/empty/771
/search/empty/772
/search/empty/778
/search/empty/917
/search/empty/918
/search/empty/824
https://t.me/nplusone
https://vk.com/nplusone
https://ok.ru/nplus1
https://twitter.com/nplusodin
https://nplus1.ru/about
https://nplus1.ru/difficult
https://nplus1.ru/adv
https://nplus1.ru/news/2015/09/21/editor-thy-name
/search/empty/869
/search/empty/874
/search/empty/880
/search/empty/768
/search/empty/890
/search/empty/871
/search/empty/876
/search/empty/775
/search/empty/767
/search/empty/771
/search/empty/772
/search/empty/778
/search/empty/917
/search/empty/918
/search/empty/824
https://offline.nplus1.ru/
https://nplus.pro/
https://t.me/nplusone
h

Ссылок много. Но нам нужны только новости – ссылки, которые начинаются со слова `/news`. Добавим условие: будем выбирать только те ссылки, в которых есть `/news`. Создадим пустой список `urls` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [10]:
urls = []

for link in soup.find_all('a'):
#     print(link)
    if '/news' in link.get('href'):
        if 'https://' in link.get('href'):
            urls.append(link.get('href'))
urls

['https://nplus1.ru/news/2015/09/21/editor-thy-name',
 'https://nplus1.ru/news/2023/04/11/i-see-men-as-trees-walking',
 'https://nplus1.ru/news/2023/04/12/parasite-vs-parasite',
 'https://nplus1.ru/news/2023/04/12/ammonia-from-water-microdroplets',
 'https://nplus1.ru/news/2023/04/12/forest-of-obstacles',
 'https://nplus1.ru/news/2023/04/12/china-analog-dart',
 'https://nplus1.ru/news/2023/04/12/hominin-footprints',
 'https://nplus1.ru/news/2023/04/11/jwst-t-gwarfs',
 'https://nplus1.ru/news/2023/04/11/prostacyclin-therapy',
 'https://nplus1.ru/news/2023/04/11/new-Puzzle-for-Nuclear-Forces',
 'https://nplus1.ru/news/2023/04/11/roboruka-endoflex-gelsight',
 'https://nplus1.ru/news/2023/04/11/sheveluch',
 'https://nplus1.ru/news/2023/04/11/Kolbe-reaction-ressurection',
 'https://nplus1.ru/news/2023/04/11/frequency-moire',
 'https://nplus1.ru/news/2023/04/11/banana-peeling',
 'https://nplus1.ru/news/2023/04/10/neurodevelopment-SARS-CoV-2',
 'https://nplus1.ru/news/2023/04/10/stm-illusions

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `full_urls` в цикле. Посмотрим на новость с индексом 1, у вас может быть другая, новости обновляются.

In [11]:
url0 = urls[1]

page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text, 'html')
soup0

<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>Бактериальный белок подарил позвоночным чувствительное зрение</title>
<link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/AeonikPro/AeonikPro-Regular.woff2" rel="preload" type="font/woff2"/>
<link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/Spectral/Spectral-Regular.woff" rel="preload" type="font/woff2"/>
<link href="/front-build/css/main.css?id=afa8ece698346e24e0ac250f09dc6e8f" rel="stylesheet"/>
<link href="/front-build/css/app.css?id=773a0b58c8d3c33a1082b6280d9ddc6d" rel="stylesheet"/>
<link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="/site.webmanifest" rel="manifest"/>
<link color="#f26e40

В коде каждой страницы с новостью есть часть с мета-информацией: датой, именем автора и проч. Такая информация окружена тэгом `<meta></meta>`. Посмотрим:

In [12]:
soup0.find_all('meta')

[<meta charset="utf-8"/>,
 <meta content="width=device-width, initial-scale=1" name="viewport"/>,
 <meta content="#f26e40" name="msapplication-TileColor"/>,
 <meta content="#ffffff" name="theme-color"/>,
 <meta content="8c90b02c84ac3b72" name="yandex-verification"/>,
 <meta content="b419949322895fc9106e24ed01be58ac" name="pmail-verification"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" name="description"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" property="og:site_name"/>,
 <meta content="Бактериальный белок подарил позвоночным чувствительное зрение" property="og:title"/>,
 <meta content="https://minio.nplus1.ru/app-images/775680/6434649e84f4b_cover_share.jpg" property="og:image"/>,
 <meta content="https://nplus1.ru/news/2023/04/11/i-see-men-as-trees-walking" property="og:url"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" property="og:description"/>,
 <meta content="article" property="og:type"/>,

Из этого списка нам нужны части с именем автора, датой, заголовком и кратким описанием. Воспользуемся поиском по атрибуту `name`. Передадим функции `find_all()` в качестве аргумента словарь с названием и значением атрибута: 

In [13]:
soup0.find_all('meta', {'name' : 'author'}) # например, автор

[<meta content="Олег Лищук" name="author"/>]

Теперь выберем единственный элемент полученного списка (с индексом 0):

In [14]:
soup0.find_all('meta', {'name' : 'author'})[0]

<meta content="Олег Лищук" name="author"/>

Нам нужно вытащить из этого объекта `content` – имя автора. Посмотрим на атрибуты:

In [15]:
soup0.find_all('meta', {'name' : 'author'})[0].attrs

{'name': 'author', 'content': 'Олег Лищук'}

Как получить отсюда `content`? Очень просто, ведь это словарь! А доставать из словаря значение по ключу мы умеем.

In [16]:
author = soup0.find_all('meta', {'name' : 'author'})[0].get('content')
author

'Олег Лищук'

Аналогичным образом извлечем дату, заголовок и описание.

In [17]:
date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].get('content')
title = soup0.find_all('meta', {'property' : 'og:title'})[0].get('content')

In [19]:
title

'Бактериальный белок подарил позвоночным чувствительное зрение'

Теперь осталось совсем чуть-чуть. Написать готовую функцию для всех проделанных нами действий и применить ее в цикле для всех ссылок в списке `full_urls`. Напишем! Аргументом функции будет ссылка на новость, а возвращать она будет текст новости и всю необходимую информацию (дата, автор, сложность и проч.). Скопируем все строки кода выше.

In [1]:
def GetNews(url0):
    """
    Возвращает кортеж с url0, date, author, title.
    Параметры:
    
    url0 - ссылка на новость (строка).
    """
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text, 'html')
    
    try:
        author = soup0.find_all('meta', {'name' : 'author'})[0].get('content')
    except IndexError:
        author = None
    date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].get('content')
    title = soup0.find_all('meta', {'property' : 'og:title'})[0].get('content')
 
    
    return url0, date, author, title

In [2]:
help(GetNews)

Help on function GetNews in module __main__:

GetNews(url0)
    Возвращает кортеж с url0, date, author, title.
    Параметры:
    
    url0 - ссылка на новость (строка).



Уфф. Осталось применить ее в цикле. Но давайте не будем спешить: импортируем функцию `sleep` для задержки, чтобы на каждой итерации цикла, прежде чем перейти к следующей новости, Python ждал несколько секунд. Во-первых, это нужно, чтобы сайт «не понял», чтобы мы его грабим, да еще автоматически. Во-вторых, с небольшой задержкой всегда есть гарантия, что страница прогрузится (сейчас это пока не очень важно, но особенно актуально будет, когда будем обсуждать встраивание в браузер с Selenium). Приступим.

In [22]:
from time import sleep

In [23]:
len(urls)

71

In [41]:
news = [] # это будет список из кортежей, в которых будут храниться данные по каждой новости

for link in urls[20:22]:
    print(link)
    res = GetNews(link)
    news.append(res)
        
    sleep(3) # задержка в 3 секунды

https://nplus1.ru/news/2023/04/10/2023-fw-13
https://nplus1.ru/news/2023/04/10/bach-vzhhh


In [42]:
news[1]

('https://nplus1.ru/news/2023/04/10/bach-vzhhh',
 '2023-04-10',
 None,
 'Манипулятор с конвейерами в пальцах повращал мячик')

Так теперь выглядит первый элемент списка:

Импортируем `pandas` и создадим датафрейм из списка кортежей: 

In [30]:
import pandas as pd

In [43]:
df = pd.DataFrame(news)

In [44]:
df.head(11)

,0,1,2,3
0,https://nplus1.ru/news/2023/04/10/2023-fw-13,2023-04-10,Александр Войтюк,Астрономы отыскали кандидата в новый квазиспут...
1,https://nplus1.ru/news/2023/04/10/bach-vzhhh,2023-04-10,None,Манипулятор с конвейерами в пальцах повращал м...


Переименуем столбцы в базе.

In [45]:
df.columns = ['link', 'date', 'author', 'title']

In [46]:
df.head(2)

,link,date,author,title
0,https://nplus1.ru/news/2023/04/10/2023-fw-13,2023-04-10,Александр Войтюк,Астрономы отыскали кандидата в новый квазиспут...
1,https://nplus1.ru/news/2023/04/10/bach-vzhhh,2023-04-10,None,Манипулятор с конвейерами в пальцах повращал м...


Всё! Сохраняем датафрейм в файл. Для разнообразия сохраним в Excel:

In [47]:
df.to_excel('nplus-news.xlsx', index=False)